In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Metric
from tensorflow.keras import regularizers
from tensorflow.keras import models
import numpy as np
import logging
tf.get_logger().setLevel(logging.ERROR)
tf.random.set_seed(7)
from sklearn.model_selection import train_test_split

In [3]:
# Load training and test datasets.
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Divida os dados em conjuntos de treinamento e teste
validation_images, test_images, validation_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

# Standardize the data.
mean = np.mean(train_images)
stddev = np.std(train_images)
train_images = (train_images - mean) / stddev
test_images = (test_images - mean) / stddev
validation_images = (validation_images - mean) / stddev

# One-hot encode labels.
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)
validation_labels = to_categorical(validation_labels, num_classes=10)

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Object used to initialize weights.
initializer = keras.initializers.RandomUniform(
    minval=-0.1, maxval=0.1)

# Create a Sequential model.
# 784 inputs.
# Two Dense (fully connected) layers with 25 and 10 neurons.
# tanh as activation function for hidden layer.
# Logistic (sigmoid) as activation function for output layer.
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(50, activation='relu',
                       kernel_initializer=initializer,
                       bias_initializer='zeros'),
    keras.layers.Dense(10, activation='softmax',
                       kernel_initializer=initializer,
                       bias_initializer='zeros')])

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [5]:
class L2ComplexityMetric(Metric):
    def __init__(self, name='l2_complexity', **kwargs):
        super(L2ComplexityMetric, self).__init__(name=name, **kwargs)
        self.l2_loss = self.add_weight(name='l2_loss', initializer='zeros')
        self.model = None

    def update_state(self, y_true, y_pred, sample_weight=None):
        if self.model is not None:
            # Calcula a perda de complexidade L2 (regularização L2)
            l2_loss = tf.add_n([tf.nn.l2_loss(weight) for weight in self.model.trainable_weights])
            self.l2_loss.assign_add(l2_loss)

    def result(self):
        return self.l2_loss

    def reset_state(self):  # Altere o nome deste método para reset_state
        # Zera as variáveis acumuladas no início de cada época ou avaliação
        self.l2_loss.assign(0.0)

In [39]:
import tensorflow as tf
import numpy as np

def lexicographic_method(solution1, solution2, initial_tolerance=0.05, tolerance_increment=0.01):
    number_objectives = len(solution1)
    score = tf.linspace(1.0, 2.0, number_objectives)
    tolerance_rel = initial_tolerance

    while True:
        for a in range(number_objectives):
            solution1 = tf.expand_dims(solution1, axis=-1)  # Adiciona uma nova dimensão
            solution1 = tf.reshape(solution1, shape=tf.shape(solution2))  # Redimensionar solution1
            best_objective = tf.minimum(solution1[a], solution2[a])
            tolerance_abs = tolerance_rel * best_objective
            comparison = tf.abs(solution1[a] - solution2[a])

            condition = tf.greater(comparison, tolerance_abs)

            if tf.reduce_any(condition):  # Check if any element in the tensor satisfies the condition
                result = tf.constant(0.0)
                if tf.reduce_all(tf.math.equal(best_objective, solution1[a])):
                    for i in range(a + 1):
                        result += solution1[i] * score[i]
                else:
                    for i in range(a + 1):
                        result += solution2[i] * score[i]
                return result

        tolerance_rel -= tolerance_increment

# Exemplo de uso:
solution1 = tf.constant([0.1, 0.2, 0.3])
solution2 = tf.constant([0.2, 0.3, 0.4])
print(lexicographic_method(solution1, solution2))


tf.Tensor(0.1, shape=(), dtype=float32)


In [21]:
def lexicographic_method(solution1, solution2, initial_tolerance = 0.05, tolerance_increment = 0.01):

  number_objectives = len(solution1)
  score = np.linspace(1, 2, number_objectives)
  tolerance_rel = initial_tolerance

  while True:
    for a in range(number_objectives):

      best_objective = min(solution1[a], solution2[a])
      tolerance_abs = tolerance_rel * best_objective
      comparison = abs(solution1[a] - solution2[a])

      if comparison > tolerance_abs:
        result = 0
        if best_objective == solution1[a]:
          for i in range(a+1):
            result += solution1[i] * score[i]
        else:
          for i in range(a+1):
            result += solution2[i] * score[i]
        return result

    tolerance_rel -= tolerance_increment

In [22]:
class CustomLoss(losses.Loss):
    def __init__(self, weights=(1.0, 1.0), name="custom_loss"):
        super().__init__(name=name)
        self.weights = weights
        self.prev_solution = [1e5, 1e5]

    def call(self, y_true, y_pred):
      # Habilitar execução imediata
        tf.config.run_functions_eagerly(True)
        # Calcula a perda de entropia cruzada (erro).
        cross_entropy_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        # Calcula a perda de complexidade L2 (regularização).
        l2_loss = tf.add_n([tf.nn.l2_loss(weight) for weight in self.model.trainable_weights])
        current_solution = [cross_entropy_loss, l2_loss]

        loss = lexicographic_method(self.prev_solution, current_solution)

        self.prev_solution = current_solution

        return loss

    def get_config(self):
        config = super(CustomLoss, self).get_config()
        config.update({"weigths": self.weights})
        return config

In [49]:
class CustomLoss(losses.Loss):
    def __init__(self, weights=(1.0, 1.0), name="custom_loss"):
        super().__init__(name=name)
        self.weights = weights
        self.prev_solution = [1e5, 1e5]

    def call(self, y_true, y_pred):
        # Redimensionar y_true e y_pred conforme necessário
        y_true = tf.reshape(y_true, shape=(-1,))
        y_pred = tf.reshape(y_pred, shape=(-1,))

        # Calcula a perda de entropia cruzada (erro).
        cross_entropy_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        # Calcula a perda de complexidade L2 (regularização).
        l2_loss = tf.add_n([tf.nn.l2_loss(weight) for weight in self.model.trainable_weights])
        current_solution = [cross_entropy_loss, l2_loss]

        loss = self.lexicographic_method(self.prev_solution, current_solution)

        self.prev_solution = current_solution

        return loss

    @staticmethod
    def lexicographic_method(solution1, solution2, initial_tolerance=0.05, tolerance_increment=0.01):
        number_objectives = len(solution1)
        score = tf.linspace(1.0, 2.0, number_objectives)
        tolerance_rel = initial_tolerance

        while True:
            for a in range(number_objectives):
                solution1 = tf.expand_dims(solution1, axis=-1)  # Adiciona uma nova dimensão
                solution1 = tf.reshape(solution1, shape=tf.shape(solution2))  # Redimensionar solution1
                best_objective = tf.minimum(solution1[a], tf.broadcast_to(solution2, tf.shape(solution1)))  # Broadcast solution2 para a mesma forma de solution1
                tolerance_abs = tolerance_rel * best_objective
                comparison = tf.abs(solution1[a] - solution2)

                condition = tf.greater(comparison, tolerance_abs)

                if tf.reduce_any(condition):  # Verifica se algum elemento no tensor satisfaz a condição
                    result = tf.constant(0.0)
                    if tf.reduce_all(tf.math.equal(best_objective, solution1[a])):
                        for i in range(a + 1):
                            result += solution1[i] * score[i]
                    else:
                        for i in range(a + 1):
                            result += solution2[i] * score[i]
                    return result

            tolerance_rel -= tolerance_increment

    def get_config(self):
        config = super().get_config()
        config.update({"weights": self.weights})
        return config

In [50]:
# Criar uma instância da métrica personalizada
l2_metric = L2ComplexityMetric()
l2_metric.model = model  # Adicione esta linha para passar a referência do modelo para a métrica

#Instancie a classe de perda personalizada.
custom_loss = CustomLoss()
custom_loss.model = model  # Associando o modelo à função de perda.

opt = keras.optimizers.Adam()

model.compile(loss=custom_loss, optimizer = opt, metrics=['categorical_crossentropy', l2_metric])

In [51]:
training_history = model.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=10, batch_size=32, verbose=2, shuffle=True)

Epoch 1/10


ValueError: No gradients provided for any variable: (['dense/kernel:0', 'dense/bias:0', 'dense_1/kernel:0', 'dense_1/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'dense/kernel:0' shape=(784, 50) dtype=float32, numpy=
array([[ 0.08228049, -0.00920287,  0.0269047 , ..., -0.07380257,
        -0.07330585, -0.08745022],
       [ 0.03678761, -0.03372712,  0.0145086 , ...,  0.05102947,
        -0.06859805, -0.00443867],
       [-0.0242447 ,  0.03110146, -0.04223147, ..., -0.03794277,
         0.08970514, -0.09583213],
       ...,
       [-0.02232022, -0.03483128,  0.05337816, ...,  0.10515669,
        -0.03507297,  0.03099254],
       [-0.07226589,  0.06444114,  0.01036159, ...,  0.06146189,
         0.038565  , -0.06975   ],
       [ 0.0187029 ,  0.06751754, -0.07882533, ...,  0.05231233,
        -0.10643183, -0.01324123]], dtype=float32)>), (None, <tf.Variable 'dense/bias:0' shape=(50,) dtype=float32, numpy=
array([ 0.00729117, -0.02076989,  0.0190273 ,  0.01772066, -0.0050

In [11]:
import sys
prev_error, prev_complexity = sys.float_info.max, sys.float_info.max

# Treinamento com o algoritmo lexicográfico
while True:
    # Treinamento do modelo
    training_history = model.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=1, batch_size=32, verbose=2, shuffle=True)

    # Avaliação dos objetivos
    error, complexity = training_history.history['val_categorical_crossentropy'], training_history.history['val_l2_complexity']

    print(error)
    print([prev_error, prev_complexity])
    print([error[9], complexity[9]])

    # Chamar a função lexicográfica para determinar se parar o treinamento
    score = lexicographic_method([error[9], complexity[9]], [prev_error, prev_complexity])

    print(score)

    # Verificar se os objetivos satisfazem o critério de parada
    if score < 2:
        break

    # Atualizar valores anteriores
    prev_error, prev_complexity = error[9], complexity[9]

Epoch 1/10
1875/1875 - 7s - loss: 2.2218 - categorical_crossentropy: 1.8146 - l2_complexity: 763.5055 - val_loss: 2.2225 - val_categorical_crossentropy: 1.8071 - val_l2_complexity: 65.2299 - 7s/epoch - 4ms/step
Epoch 2/10
1875/1875 - 4s - loss: 2.2221 - categorical_crossentropy: 1.8143 - l2_complexity: 764.6462 - val_loss: 2.2210 - val_categorical_crossentropy: 1.8131 - val_l2_complexity: 64.0401 - 4s/epoch - 2ms/step
Epoch 3/10
1875/1875 - 5s - loss: 2.2219 - categorical_crossentropy: 1.8143 - l2_complexity: 764.3713 - val_loss: 2.2141 - val_categorical_crossentropy: 1.8047 - val_l2_complexity: 64.2731 - 5s/epoch - 3ms/step
Epoch 4/10
1875/1875 - 4s - loss: 2.2216 - categorical_crossentropy: 1.8132 - l2_complexity: 765.6340 - val_loss: 2.2162 - val_categorical_crossentropy: 1.8061 - val_l2_complexity: 64.3736 - 4s/epoch - 2ms/step
Epoch 5/10
1875/1875 - 4s - loss: 2.2209 - categorical_crossentropy: 1.8141 - l2_complexity: 762.7682 - val_loss: 2.2134 - val_categorical_crossentropy: 1.7

In [10]:
# Calculate the error and norm L2 on the test set
metrics = model.evaluate(test_images, test_labels)

157/157 [==============================] - 0s 2ms/step - loss: 2.2138 - categorical_crossentropy: 1.8090 - l2_complexity: 63.5507
